In [1]:
!curl https://raw.githubusercontent.com/Vallisha-M/ML-Lab/main/Naive-Bayes-Classifier/dataset.csv > dataset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   419  100   419    0     0   3325      0 --:--:-- --:--:-- --:--:--  3325


In [12]:
import pandas as pd

In [50]:
dataset = pd.read_csv('dataset.csv')

In [51]:
dataset

,outlook,temperature,humidity,wind,playtennis
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [52]:
m=dataset.shape[0]

In [53]:
table_res={}

for i in range(m):
  if dataset.iloc[i][-1] in table_res:
    table_res[dataset.iloc[i][-1]].append(i)
  else:
    table_res[dataset.iloc[i][-1]]=[i]

In [54]:
table_res

{'no': [0, 1, 5, 7, 13], 'yes': [2, 3, 4, 6, 8, 9, 10, 11, 12]}

In [55]:
table_cols={}
for col in dataset.columns:
  table_cols[col]={}
  for i in range(m):
    if dataset.iloc[i][col] in table_cols[col]:
      table_cols[col][dataset.iloc[i][col]]+=1
    else:
      table_cols[col][dataset.iloc[i][col]]=1

In [56]:
table_cols

{'humidity': {'high': 7, 'normal': 7},
 'outlook': {'overcast': 4, 'rain': 5, 'sunny': 5},
 'playtennis': {'no': 5, 'yes': 9},
 'temperature': {'cool': 4, 'hot': 4, 'mild': 6},
 'wind': {'strong': 6, 'weak': 8}}

In [57]:
table={}
for col in dataset.columns[:-1]:
  table[col]={}
  for res in table_res:
    table[col][res]={}
    for value in table_cols[col]:
      table[col][res][value]=0

In [58]:
table

{'humidity': {'no': {'high': 0, 'normal': 0}, 'yes': {'high': 0, 'normal': 0}},
 'outlook': {'no': {'overcast': 0, 'rain': 0, 'sunny': 0},
  'yes': {'overcast': 0, 'rain': 0, 'sunny': 0}},
 'temperature': {'no': {'cool': 0, 'hot': 0, 'mild': 0},
  'yes': {'cool': 0, 'hot': 0, 'mild': 0}},
 'wind': {'no': {'strong': 0, 'weak': 0}, 'yes': {'strong': 0, 'weak': 0}}}

In [59]:
for col in dataset.columns[:-1]:
  for res in table_res:
    for i in table_res[res]:
      col_value=dataset.iloc[i][col]
      added=float(1/table_cols[col][col_value])
      table[col][res][col_value]+=added

In [60]:
table

{'humidity': {'no': {'high': 0.5714285714285714,
   'normal': 0.14285714285714285},
  'yes': {'high': 0.42857142857142855, 'normal': 0.857142857142857}},
 'outlook': {'no': {'overcast': 0, 'rain': 0.4, 'sunny': 0.6000000000000001},
  'yes': {'overcast': 1.0, 'rain': 0.6000000000000001, 'sunny': 0.4}},
 'temperature': {'no': {'cool': 0.25, 'hot': 0.5, 'mild': 0.3333333333333333},
  'yes': {'cool': 0.75, 'hot': 0.5, 'mild': 0.6666666666666666}},
 'wind': {'no': {'strong': 0.5, 'weak': 0.25},
  'yes': {'strong': 0.5, 'weak': 0.75}}}

In [61]:
def predict(instance, table):
  table_answer={}
  for res in table_res:
    table_answer[res]=1
    for col in instance:
      table_answer[res]*=table[col][res][instance[col]]
  ans_key=-1
  ans_val=-1
  for res in table_answer:
    if table_answer[res]>ans_val:
      ans_val=table_answer[res]
      ans_key=res
  return ans_key


In [62]:
predict({"temperature":"hot","outlook":"sunny","wind":"weak","humidity":"high"}, table)

'yes'

In [69]:
def test(path, table):
  data=pd.read_csv(path)
  result={"correct":0,"incorrect":0, "confusion_matrix":{"tp":0,'tn':0,"fp":0,"fn":0}}

  for i in range(data.shape[0]):
    instance={}
    for col in data.columns[:-1]:
      instance[col]=data.iloc[i][col]
    predicted=predict(instance, table)
    if predicted==data.iloc[i][-1]:
      result['correct']+=1
      if predicted=='no':
        result['confusion_matrix']['tn']+=1
      else:
        result['confusion_matrix']['tp']+=1

    else:
      result['incorrect']+=1
      if predicted=='no':
        result['confusion_matrix']['fn']+=1
      else:
        result['confusion_matrix']['fp']+=1
  result['accuracy']=result['correct']/(result['correct']+result['incorrect'])
  result['precision']=result['confusion_matrix']['tp']/(result['confusion_matrix']['tp']+result['confusion_matrix']['fp'])
  result['recall']=result['confusion_matrix']['tp']/(result['confusion_matrix']['tp']+result['confusion_matrix']['fn'])
  result['f1']=2*result['precision']*result['recall']/(result['precision']+result['recall'])
  return result

In [70]:
test('dataset.csv', table)

{'accuracy': 0.7142857142857143,
 'confusion_matrix': {'fn': 0, 'fp': 4, 'tn': 1, 'tp': 9},
 'correct': 10,
 'f1': 0.8181818181818181,
 'incorrect': 4,
 'precision': 0.6923076923076923,
 'recall': 1.0}

##As class

In [8]:
import pandas as pd
class NaiveBayes:
  def __init__(self, path:str):
    self.getTable(path)
    
  def getTable(self,path):
    dataset = pd.read_csv(path)
    m=dataset.shape[0]

    table_res={}
    for i in range(m):
      if dataset.iloc[i][-1] in table_res:
        table_res[dataset.iloc[i][-1]].append(i)
      else:
        table_res[dataset.iloc[i][-1]]=[i]
    self.table_res=table_res
    table_cols={}
    for col in dataset.columns:
      table_cols[col]={}
      for i in range(m):
        if dataset.iloc[i][col] in table_cols[col]:
          table_cols[col][dataset.iloc[i][col]]+=1
        else:
          table_cols[col][dataset.iloc[i][col]]=1
    self.table={}
    for col in dataset.columns[:-1]:
      self.table[col]={}
      for res in table_res:
        self.table[col][res]={}
        for value in table_cols[col]:
          self.table[col][res][value]=0
    
    for col in dataset.columns[:-1]:
      for res in table_res:
        for i in table_res[res]:
          col_value=dataset.iloc[i][col]
          added=float(1/table_cols[col][col_value])
          self.table[col][res][col_value]+=added
  def predict(self,instance):
    table_answer={}
    for res in self.table_res:
      table_answer[res]=1
      for col in instance:
        table_answer[res]*=self.table[col][res][instance[col]]
    ans_key=-1
    ans_val=-1
    for res in table_answer:
      if table_answer[res]>ans_val:
        ans_val=table_answer[res]
        ans_key=res
    return ans_key
  def test(self,path):
    data=pd.read_csv(path)
    result={"correct":0,"incorrect":0, "confusion_matrix":{"tp":0,'tn':0,"fp":0,"fn":0}}

    for i in range(data.shape[0]):
      instance={}
      for col in data.columns[:-1]:
        instance[col]=data.iloc[i][col]
      predicted=self.predict(instance)
      if predicted==data.iloc[i][-1]:
        result['correct']+=1
        if predicted=='no':
          result['confusion_matrix']['tn']+=1
        else:
          result['confusion_matrix']['tp']+=1
      else:
        if predicted=='no':
          result['confusion_matrix']['fn']+=1
        else:
          result['confusion_matrix']['fp']+=1
        result['incorrect']+=1

    result['accuracy']=result['correct']/data.shape[0]
    result['precision']=result['confusion_matrix']['tp']/(result['confusion_matrix']['tp']+result['confusion_matrix']['fp'])
    result['recall']=result['confusion_matrix']['tp']/(result['confusion_matrix']['tp']+result['confusion_matrix']['fn'])
    result['f1']=2*result['precision']*result['recall']/(result['precision']+result['recall'])

    return result

In [9]:
naiveBayes=NaiveBayes('dataset.csv')

In [10]:
naiveBayes.test('dataset.csv')

{'accuracy': 0.7142857142857143,
 'confusion_matrix': {'fn': 0, 'fp': 4, 'tn': 1, 'tp': 9},
 'correct': 10,
 'f1': 0.8181818181818181,
 'incorrect': 4,
 'precision': 0.6923076923076923,
 'recall': 1.0}